In [1]:
import requests
import base64
import datetime

In [2]:
client_id = '********************'
client_secret = '********************'

In [8]:

class SpotifyAPI(object):
    """
    Spotify Client
    
    Parameters
    ---
    client_id : str
    client_secret : str
    """
    token_url = "https://accounts.spotify.com/api/token"
    access_token = None
    access_token_expires = datetime.datetime.now()
    client_id = None
    client_secret = None

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns :  a base 64 encoded string.
        """
        if self.client_id is None or self.client_secret is None:
            raise Exception("You must set client_id and client_secret")
        client_cred = f"{self.client_id}:{self.client_secret}"
        client_cred_b64 = base64.b64encode(client_cred.encode())
        return client_cred_b64.decode()

    def get_token_headers(self):
        client_cred_b64 = self.get_client_credentials()
        return {
            'Authorization': f'Basic {client_cred_b64}'}

    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
        }

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_header = self.get_token_headers()
        r = requests.post(token_url,
                          data=token_data,
                          headers=token_header)
        if r.status_code not in range(200, 299):
            raise Exception('Could not authenticate client')
        data = r.json()
        now = datetime.datetime.now()
        self.access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            return self.perform_auth()
        elif token is None:
            self.perform_auth()
            return self.get_access_token()
        return token

    def search(self, query, search_type='artist'):
        from urllib.parse import urlencode
        access_token = self.access_token
        headers = self.get_resource_header()
        endpoint = 'https://api.spotify.com/v1/search'
        data = urlencode({"q": query, "type": search_type})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    def get_resource_header(self):
        access_token = self.access_token
        headers = {
            'Authorization' : f"Bearer {access_token}"
            }
        return headers


    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
             return {}
        return r.json()

    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')


    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')



In [9]:
spotify = SpotifyAPI(client_id, client_secret)

In [10]:
spotify.perform_auth()

True

In [11]:
spotify.search(query='time', search_type='track')

200


b67616d000048519dcf31d6b5eb82fa24de783a',
       'width': 64}],
     'name': 'One-X',
     'release_date': '2006-06-25',
     'release_date_precision': 'day',
     'total_tracks': 12,
     'type': 'album',
     'uri': 'spotify:album:4ZDwEvXIKxWdwhAQK8ndBU'},
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2xiIXseIJcq3nG7C8fHeBj'},
      'href': 'https://api.spotify.com/v1/artists/2xiIXseIJcq3nG7C8fHeBj',
      'id': '2xiIXseIJcq3nG7C8fHeBj',
      'name': 'Three Days Grace',
      'type': 'artist',
      'uri': 'spotify:artist:2xiIXseIJcq3nG7C8fHeBj'}],
    'available_markets': ['AD',
     'AE',
     'AL',
     'AR',
     'AU',
     'BA',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'BY',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FR',
     'GB',
     'GR',
     'GT',
     'HK',
     'HN',
     'HR',
     'HU',
   

In [12]:
spotify.search('kumar sanu', 'artist')

200


{'artists': {'href': 'https://api.spotify.com/v1/search?query=kumar+sanu&type=artist&offset=0&limit=20',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4K6blSRoklNdpw4mzLxwfn'},
    'followers': {'href': None, 'total': 2547918},
    'genres': ['classic bollywood',
     'desi pop',
     'filmi',
     'modern bollywood',
     'sufi'],
    'href': 'https://api.spotify.com/v1/artists/4K6blSRoklNdpw4mzLxwfn',
    'id': '4K6blSRoklNdpw4mzLxwfn',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/b802604a568d80882fb4e0ed5862f52aeed4c2fa',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/07396382e84791f83505efb6bf9934823443fd18',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/05fbe4fe546ca10b31f18b621138f7a90d52c01f',
      'width': 160}],
    'name': 'Kumar Sanu',
    'popularity': 67,
    'type': 'artist',
    'uri': 'spotify:artist:4K6blSRoklNdpw4mzLxwfn'},
   {'external_urls': {'sp

In [13]:
spotify.get_artist('4K6blSRoklNdpw4mzLxwfn')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4K6blSRoklNdpw4mzLxwfn'},
 'followers': {'href': None, 'total': 2547918},
 'genres': ['classic bollywood',
  'desi pop',
  'filmi',
  'modern bollywood',
  'sufi'],
 'href': 'https://api.spotify.com/v1/artists/4K6blSRoklNdpw4mzLxwfn',
 'id': '4K6blSRoklNdpw4mzLxwfn',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/b802604a568d80882fb4e0ed5862f52aeed4c2fa',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/07396382e84791f83505efb6bf9934823443fd18',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/05fbe4fe546ca10b31f18b621138f7a90d52c01f',
   'width': 160}],
 'name': 'Kumar Sanu',
 'popularity': 67,
 'type': 'artist',
 'uri': 'spotify:artist:4K6blSRoklNdpw4mzLxwfn'}

In [16]:
spotify.get_album("2xiIXseIJcq3nG7C8fHeBj")

{}